# Du-Proprio ML Project
### Data Innovation Society
#### Matteo Esposito, Victor Debray, Edric Jung

> TODO
* Preprocess Data for Geocoding (fix wrongly formatted addresses)
* Geocode (find postal codes using addresses)
* EDA
* Model

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

In [3]:
# Load data
df = pd.read_csv(os.getcwd() + '/du_proprio_scraped_FULL.csv')

## Data Preprocessing

In [30]:
def displayAll(df):
    '''
    Function to ensure exhaustive output of table information. 
    '''
    with pd.option_context('display.max_rows', 2000):
        with pd.option_context('display.max_columns', 2000):
            display(df)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19701 entries, 0 to 19798
Data columns (total 8 columns):
Rent Price            19701 non-null object
Address               19701 non-null object
Location              19701 non-null object
Number of Bed         19701 non-null int64
Number of Bath        19701 non-null object
Living Area (ft^2)    19701 non-null int64
Land Area (ft^2)      19701 non-null int64
respects_dt_format    19701 non-null bool
dtypes: bool(1), int64(3), object(4)
memory usage: 1.2+ MB


In [32]:
# Removing rows with NAN addresses = 98/19799 rows.
df = df[pd.notnull(df['Address'])]

In [6]:
# Get an idea of what type of addresses we are dealing with
df['Address'].value_counts()

Modèle à construire - Par les Constructions André Jacques inc.    4
chemin des Hauteurs                                               4
Montée du Cerf                                                    4
135, rue Principale                                               3
Chemin des Cerisiers                                              3
2605, chemin Ste-Foy                                              3
285, RUE LABERGE                                                  3
Le Gardenville - Unité 2-111                                      3
12370, rue Jean-Baptiste-Normand                                  3
rue Kildare - Par Construction C.R.D                              3
4560, rue Bosco                                                   3
1320, AV DES MELEZES                                              3
, Domaine Lévesque                                                3
rue Bellevue - Par Construction C.R.D                             3
À construire - Par Gestion C.Clermont           

In [7]:
# Define boolean variable to flag entries/addresses that do not respect the digit + "," + char format
# Doing this to be able to work fluidly with postal code API
df['respects_dt_format'] = df.Address.str.contains('([0-9]+,)(.*[a-zA-Z]+)')

In [25]:
# Create table of bad addresses
df_unformatted = df[df['respects_dt_format'] == False]
print('% of wrongly formatted rows: {} %'.format(round(100*len(df_unformatted)/len(df),3)))

% of wrongly formatted rows: 6.812 %


In [33]:
displayAll(df_unformatted['Address'].value_counts())

chemin des Hauteurs                                                                                       4
Montée du Cerf                                                                                            4
Modèle à construire - Par les Constructions André Jacques inc.                                            4
Chemin des Cerisiers                                                                                      3
Projet domiciliaire Le Majordome - Chemin James-McFaul                                                    3
rue Kildare - Par Construction C.R.D                                                                      3
À construire - Par Gestion C.Clermont                                                                     3
chemin Abercrombie                                                                                        3
rang Sainte-Catherine                                                                                     3
rue Bellevue - Par Construct